In [ ]:
from __future__ import annotations
from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import IAMTokenAuth

import dotenv, os

dotenv.load_dotenv()

config = dotenv.dotenv_values()

messages = [
    {
        "role": "system",
        "text": "Отвечай на вопрос максимально правдиво",
    },
]


def main():
    sdk = YCloudML(
        folder_id=config["YC_FOLDER_ID"],
        auth=IAMTokenAuth(config["YC_IAM_TOKEN"]),
        enable_server_data_logging=False,
    )
    model = sdk.models.completions("yandexgpt").configure(temperature=0)
    print("Начало чата:")

    msg_text = input("> ")
    if msg_text.lower() == "exit":
            return
    messages.append({"role": "user", "text": msg_text})
    result = (
        model.run(messages)
    )


    for alternative in result:
        print(f"YaGPT> {alternative.text}")

    while True:
        msg_text = input("> ")
        if msg_text.lower() == "exit":
            return
        messages.append({"role": "user", "text": msg_text})
        result = (
            model.run(messages)
        )
        messages.append({"role": result.role, "text": result.text})
        print(f"YaGPT> {result.text}")

if __name__ == "__main__":
    main()

Начало чата:


>  Какая погода была в Москве 12 Мая 2025 года?


YaGPT> Я не могу предоставить информацию о погоде в конкретный день в будущем, так как у меня нет доступа к данным о погоде наперёд. Чтобы узнать погоду в Москве 12 мая 2025 года, вам нужно будет проверить прогноз погоды ближе к этой дате.


>  Какая погода была в Москве 12 Мая 2021 года?


YaGPT> К сожалению, я не могу предоставить информацию о погоде в Москве на конкретные даты в 2021 и 2025 годах, так как у меня нет доступа к историческим данным о погоде. Рекомендую обратиться к архивным источникам или специализированным сервисам для получения более точной информации.


...

YaGPT> По данным различных источников, 12 мая 2021 года в Москве была переменная облачность. Температура воздуха достигала +19 °C.


>  Какая погода была в Москве 12 Мая 2025 года?


YaGPT> К сожалению, у меня нет информации о погоде в Москве на 12 мая 2025 года.


>  Какая погода была в Москве 12 Мая 2024 года?


YaGPT> По данным различных источников, 12 мая 2024 года в Москве была переменная облачность. Температура воздуха достигала +21 °C.


In [ ]:
Потоковая генерация:

In [3]:
from __future__ import annotations
from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import IAMTokenAuth

import dotenv, os

dotenv.load_dotenv()

config = dotenv.dotenv_values()

messages = [
    {"role": "system", "text": "Найди ошибки в тексте и исправь их"},
    {"role": "user", "text": "Ашипки саме сибя ни исрпвят."},
    {"role":"assistant","text":"Ошибки сами себя не исправят."},
]


def main():
    sdk = YCloudML(
        folder_id=config["YC_FOLDER_ID"],
        auth=IAMTokenAuth(config["YC_IAM_TOKEN"]),
        enable_server_data_logging=False,
    )

    model = sdk.models.completions("yandexgpt-lite")

    
    for result in model.configure(temperature=0.5).run_stream(messages):
        for alternative in result:
            print(alternative)
    

if __name__ == "__main__":
    main()
    
    


Alternative(role='assistant', text='О', status=<AlternativeStatus.PARTIAL: 1>, tool_calls=None)
Alternative(role='assistant', text='Ошибки сами себя не исправляют.', status=<AlternativeStatus.FINAL: 3>, tool_calls=None)


In [ ]:
Асинхронная генерация:

In [2]:
from __future__ import annotations
import time
from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import IAMTokenAuth

import dotenv, os

dotenv.load_dotenv()

config = dotenv.dotenv_values()

messages_1 = [
    {
        "role": "system",
        "text": "Найди ошибки в тексте и исправь их",
    },
    {
        "role": "user",
        "text": """Ламинат подойдет для укладке на кухне или в детской 
комнате – он не боиться влаги и механических повреждений благодаря 
защитному слою из облицованных меламиновых пленок толщиной 0,2 мм и 
обработанным воском замкам.""",
    },
]

messages_2 = [
    {"role": "system", "text": "Найди ошибки в тексте и исправь их"},
    {"role": "user", "text": "Ашипки саме сибя ни исрпвят."},
]


def main():

    sdk = YCloudML(
        folder_id=config["YC_FOLDER_ID"],
        auth=IAMTokenAuth(config["YC_IAM_TOKEN"]),
        enable_server_data_logging=False,
    )

    model = sdk.models.completions("yandexgpt-lite")

    # Variant 1: подождать выполнения операции, используя 5мс периоды time.sleep(5)

    print("Variant 1:")

    operation = model.configure(temperature=0.5).run_deferred(messages_1)

    status = operation.get_status()
    while status.is_running:
        time.sleep(5)
        status = operation.get_status()

    result = operation.get_result()
    print(result)

    # Variant 2: подождать выполнения операции через метод wait()

    print("Variant 2:")

    operation = model.run_deferred(messages_2)

    result = operation.wait()
    print(result)


if __name__ == "__main__":
    main()


Variant 1:
GPTModelResult(alternatives=(Alternative(role='assistant', text='Ламинат подойдёт для укладки на кухне или в детской комнате — он не боится влаги и механических повреждений благодаря защитному слою из облицованных меламиновых плёнок толщиной 0,2 мм и обработанным воском замкам.', status=<AlternativeStatus.FINAL: 3>, tool_calls=None),), usage=CompletionUsage(input_text_tokens=74, completion_tokens=46, total_tokens=120, reasoning_tokens=0), model_version='23.10.2024')
Variant 2:
GPTModelResult(alternatives=(Alternative(role='assistant', text='Ошибки сами себя не исправят.', status=<AlternativeStatus.FINAL: 3>, tool_calls=None),), usage=CompletionUsage(input_text_tokens=32, completion_tokens=8, total_tokens=40, reasoning_tokens=0), model_version='23.10.2024')
